# Generic tasks using the [FAA Releasable Database](https://www.faa.gov/licenses_certificates/aircraft_certification/aircraft_registry/releasable_aircraft_download/?utm_source=tech.mazavr.tk&amp;utm_medium=link&amp;utm_compaign=article)

Note: it is a good idea to download the database from time to time to stay abreast of updates.

In [ ]:
import os
import glob
import pyodbc
import numpy as np
import pandas as pd
import geopandas as gpd
import rasterio
import rasterio.plot
import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d

# def Type1_Sites():
    
#     import pyodbc

#     # set up a connection to the database
#     cnxn = pyodbc.connect("Driver={SQL Server};"
#                           r"Server=inp2300irmadb01\irma;"
#                           "Trusted_Connection=Yes;"
#                           "Database=NSNSD_Acoustic;")

#     cursor = cnxn.cursor() # establish cursor

#     # /////// load data into a dataframe ///////////   
#     sql = "SELECT ID, UnitID, Code, Name, LAT, LON FROM Site"
#     metadata = pd.read_sql(sql,cnxn)  # without parameters [non-prepared statement]

#     # close both the cursor and connection    
#     cursor.close()
#     cnxn.close()

#     return metadata

# # let's just load the metadata by default
# sites = Type1_Sites()

# load the FAA Releasable Database
# fair warning: IT IS BIG
FAA = pd.read_csv(r"V:\Noncanonical Data\ReleasableAircraft\MASTER.txt", sep=",")
Type_Map = {4:"Fixed-wing", 5:"Jet", 6:"Helicopter"} # codifed type to human-readable type
Color_Map = {0:"gray", 1:"red", 2:"skyblue"}

FLY

'N185AR' 'N907KW'

K2A

'N727KT' 'N929KT' 'N636KT' 'N121KT' 'N320KT' 'N424KT' 'N343KT' 'N626KT' 'N242KT' 'N535KT'

KAT

'N72309' 'N619CH' 'N72395' 'N74PS'

NPS

'N21HY' 'N473YC' 'N192CC' 'N709M' 'N706M' 'N534L' 'N278CC' 'N1004' 'N9871D' 'N570AE' 'N970TH' 'N644TC' 'N567L' 'N446TC' 'N149AE' 'N437CC'

SHL

'N121EE' 'N2749' 'N677TX'

TAT

'N100BW' 'N185FK' 'N80945' 'N510PR' 'N565TA' 'N88765' 'N2YV'

### 2022 08 18 look up N-number given a Mode S Hex ID
<font color="green" size=5>This is probably what you want</font>

In [ ]:
# enter a HexID (or list of HexIDs)
# HexIDs = np.array(["A7EA0", "A84B83", "AB099C", "AC8A23", "ADD2C4", "AE5A22", "AA36FF", "AD92C4", "AD92C4"])
# HexIDs = np.array(["AD92C4"])
# HexIDs = np.array(['AE57A0', 'A980D8', 'A9E799', 'A7D9F3', 'A8752D', 'A78EB5','A534E7', 'A7D293']) # just the IDs that the ATMP may have changed...

# # this is all the MORU aircraft observed by ADS-B pre or post!
# HexIDs = np.array(['0266ED', '07B949', '763F00', '7F2834', 'A001D4', 'A00A5F',
#                    'A00C82', 'A03746', 'A03E69', 'A045BC', 'A045DD', 'A04B68',
#                    'A04BA8', 'A04C72', 'A07FED', 'A0809A', 'A09901', 'A0B316',
#                    'A0D32E', 'A0DC0A', 'A103D5', 'A126DE', 'A1490F', 'A14B8F',
#                    'A15AB9', 'A189B3', 'A190D7', 'A19EA5', 'A21DA5', 'A242D2',
#                    'A2457D', 'A293CE', 'A29BB1', 'A2EE54', 'A2F7D3', 'A300AB',
#                    'A32234', 'A34127', 'A344EC', 'A3C552', 'A3FD02', 'A445A2',
#                    'A46456', 'A47623', 'A47861', 'A4F4C8', 'A4F9BA', 'A4FA4C',
#                    'A502B4', 'A505BA', 'A511E7', 'A534E7', 'A54EAF', 'A56433',
#                    'A57606', 'A58984', 'A5999F', 'A5BEE3', 'A5D1EB', 'A5DEE6',
#                    'A5EABF', 'A621E9', 'A6538C', 'A65B6B', 'A65BD3', 'A67197',
#                    'A68077', 'A6809E', 'A6BD23', 'A6DE55', 'A73D55', 'A73FB9',
#                    'A76780', 'A78EB5', 'A78EFB', 'A7A778', 'A7D293', 'A7D9F3',
#                    'A80112', 'A80649', 'A80870', 'A81836', 'A83D1E', 'A84F38',
#                    'A856F8', 'A85BB4', 'A862B7', 'A8752D', 'A875F4', 'A8B8F0',
#                    'A8B90A', 'A8FD11', 'A900FA', 'A92982', 'A96851', 'A96A38',
#                    'A980D8', 'A99616', 'A9C9CD', 'A9CE51', 'A9E5BA', 'A9E799',
#                    'AA049F', 'AA30CF', 'AA376D', 'AA3F44', 'AA486F', 'AAB1A9',
#                    'AACF97', 'AAFB43', 'AB2F9D', 'AB3609', 'AB9119', 'ABA61E',
#                    'ABA887', 'ABCD81', 'ABCE19', 'ABE554', 'AC1C46', 'AC2F1C',
#                    'AC9C00', 'ACF10E', 'ACF5D2', 'AD7D80', 'ADA090', 'ADAE4D',
#                    'AE063F', 'AE20B9', 'AE4790', 'AE54CA', 'AE57A0', 'AE5CAF',
#                    'AE68B7', 'AE6BAE', 'C03CC3', 'CF1E00'])


HexIDs = np.array(['AD876E'])

FAA['MODE S CODE HEX'] = FAA['MODE S CODE HEX'].astype('str').str.strip()
FAA_lookup = FAA.loc[FAA['MODE S CODE HEX'].isin(HexIDs), ['N-NUMBER', 'TYPE AIRCRAFT', "NAME", "MODE S CODE HEX"]]
FAA_lookup['TYPE AIRCRAFT'] = FAA_lookup['TYPE AIRCRAFT'].apply(lambda l: Type_Map[int(l)])
FAA_lookup

# %store FAA_lookup
# FAA.columns

### 2022 08 19 Given a list of N-Numbers, look up information
<font color="green" size=3>This is also handy</font>

In [ ]:
N_numbers = np.array(['N2YV', 'NAC1781', 'N125KT', 'N570AE', 'N424KT', 'N565TA',
           'N100BW', 'N929KT', 'N166SB', 'N636KT', 'N4377U', 'N44587',
           'N343KT', 'N453SP', 'N55169', 'N727KT', 'N981BB', 'N88765',
           'N1598H', 'N510PR', 'N1748R', 'N93SF', 'N574HB', 'N303GV',
           'N641AT', 'N546LM', 'N317KC', 'N731E', 'N47698', 'N974CS',
           'NAC760', 'N320KT', 'N8032A', 'N121KT', 'N53847', 'N548LM',
           'N542LM', 'NAC765', 'N408SR', 'NAC722', 'NAC720', 'NAC723',
           'N1292F', 'N440RA', 'NAC721', 'N92AG', 'N3349S', 'N1543', 'N2740X',
           'N62LC', 'N9939N', 'N80945', 'N4444Z', 'N734KF', 'NAC753',
           'N6728H', 'N1797R', 'N626KT', 'N8004N', 'N905AK', 'N27917',
           'N145XL', 'N411WA', 'N52168', 'NAC1782', 'N102U', 'N9508W',
           'NAC1783', 'N1726R', 'N46551', 'N607FJ', 'N21709', 'N865HP',
           'N232KT', 'N4661Z', 'N51436', 'N27663', 'N115U', 'N2879F',
           'N916TL', 'N68MN', 'N70901', 'NCR653', 'N7136Q', 'N455Z', 'N98SP',
           'NCA133', 'N707H', 'N97705', 'N541LM', 'NORTH47', 'N410SP',
           'N32PX', 'N122KT', 'N185FK', 'NAC725', 'NCA051', 'N715TL',
           'N630PM', 'N9188M', 'N7595Q', 'N774AK', 'NAC762', 'NAC763',
           'N944JD', 'NAC2720', 'N1779J', 'N126EC', 'N208W', 'N91AC',
           'NAC1780', 'NAC2721', 'N646AT', 'N98354', 'NORTH48', 'N222GY',
           'N5227R', 'N45052', 'N558LM', 'NAC732', 'N907KW', 'N757GQ',
           'N479AK', 'N757', 'N5600B', 'N915NG', 'N541RL', 'N53AR', 'N73605',
           'N6345Z', 'N4906N', 'N158RX', 'N3083R', 'N3085Q', 'N9877R',
           'N1867', 'N21132', 'N70052', 'N521GV', 'N47915', 'NAC2765',
           'N724BB', 'N6531N', 'N645AT', 'N1018B', 'N5506J', 'N291DT',
           'N757ZN', 'N9951N', 'NAC726', 'NAC730', 'N3602Y', 'N4260L',
           'N9100H', 'NAC727', 'N52654', 'N418SP', 'N777CQ', 'NAC766',
           'N2366V', 'NAC767', 'N185AH', 'N708RE', 'N126AK', 'N16930',
           'N644TC', 'N7748H', 'N284PM', 'N531LM', 'NAC2753', 'N9002H',
           'NAC2762', 'NAC2723', 'N9885Q', 'NAC2763', 'NAC734', 'N827HB',
           'NAC752', 'N4948Q', 'N916Y', 'NAC2726', 'N2848A', 'NAC2727',
           'N970TH', 'N500D', 'N4891Z', 'N90', 'N881PR', 'N41GW', 'N13205',
           'N2561X', 'N94598', 'N8355Q', 'N2558L', 'N654', 'N75GE', 'N67178',
           'N924AC', 'N2624X', 'N5700M', 'N1820Z', 'N13AL', 'N715SJ',
           'NAC2722', 'N298B', 'N963CE', 'N52197', 'N100AK', 'N33JH',
           'NCA004', 'N756LC', 'N695AK', 'N8615Y', 'N3054V', 'N1733R',
           'N185XL', 'N880DC', 'N4649U', 'N1074G', 'N26GB', 'N9209H',
           'N619CH', 'N217EB', 'NAC2760', 'N1525C', 'N109JG', 'N369P',
           'N449MM', 'N5001R', 'N962CE', 'N733JP', 'N7987V', 'N792SP',
           'N381AK', 'N907YK', 'N907TK', 'N6LU', 'N5AR', 'N3974X', 'N9044M',
           'N161RD', 'N138VB', 'N43BV', 'N33NE', 'N456SF', 'N50467', 'N4288M',
           'N557LM', 'NCA5011', 'N904AK', 'N117AM', 'N518MF', 'N123SH',
           'N840AK', 'N53PA', 'N35339', 'N560RP', 'N756PF', 'N7MZ', 'N1926Q',
           'N5339H', 'N351SA', 'N555FS', 'N375SP', 'N19805', 'N2069B', 'N86',
           'N587JF', 'N9776R', 'N959UA', 'N6598B', 'N59KL', 'N747FL',
           'N2899J', 'N70199', 'N949BW', 'N111NS', 'N11AK', 'N65065',
           'N7848B', 'N1234', 'N533LM', 'N60529', 'N2805K', 'N530LM',
           'N49827', 'N5275E', 'N9205D', 'N696SM', 'NAC2752', 'N18NT',
           'N707RA', 'N750PM', 'N111LP', 'N840ER', 'N41101', 'N3455Y',
           'N2885C', 'N317AK', 'N829RA', 'N55WJ', 'N816SW', 'N222ME',
           'N741RS', 'N715HL', 'N257LM', 'N203DH', 'N4593V', 'NCA167',
           'N2921A'], dtype=object)

FAA_lookup = FAA.loc[FAA["N-NUMBER"].isin(pd.Series(N_numbers).str[1:].values), ['N-NUMBER', 'TYPE AIRCRAFT', "NAME", "MODE S CODE HEX"]]
FAA_lookup['TYPE AIRCRAFT'] = FAA_lookup['TYPE AIRCRAFT'].apply(lambda l: Type_Map[int(l)])
FAA_lookup #.loc[FAA_lookup["TYPE AIRCRAFT"] == "Helicopter", "N-NUMBER"].values
FAA_lookup.sort_values("NAME")

### 2022 08 19 Using a *.geojson* from `NPS-ActiveSpace`, look up aircraft information for audible flights

In [ ]:
gdf = gpd.read_file(r"C:\Users\DBetchkal\Desktop\DENATRLA2021_saved_annotations_CANTWELL_ADSB.geojson")
HexIDs = gdf.loc[gdf["audible"] == '1', "_id"].apply(lambda s: s.split("_")[0])
FAA['MODE S CODE HEX'] = FAA['MODE S CODE HEX'].astype('str').str.strip()
FAA_lookup = FAA.loc[FAA['MODE S CODE HEX'].isin(HexIDs), ['N-NUMBER', 'TYPE AIRCRAFT', "NAME", "MODE S CODE HEX"]]
FAA_lookup['TYPE AIRCRAFT'] = FAA_lookup['TYPE AIRCRAFT'].apply(lambda l: Type_Map[int(l)])
FAA_lookup.sort_values("NAME")

### 2022 08 19 interactive 3D plot for reviewing annotations
<font color="orange">**Note:** I've been able to load no more than X annotations successfully.</font>

In [ ]:
# where is the annotation work?
annotations = r"V:\NMSim\01 SITES\DENAMOOS\DENAMOOS2018_saved_annotations.geojson"

# load the study area 
# we'll use it to set the plot extent in the xy-plane
des = os.path.basename(annotations).split("_")[0]
study_area = gpd.read_file(os.path.join(r"V:\NMSim\01 SITES", des[:-4]))
xmin, ymin, xmax, ymax = study_area.total_bounds

site_coords = sites.loc[sites["Code"] == des[:-4], ["LON", "LAT"]].values

# find the elevation .tif
elev_path = glob.glob(os.path.join(r"V:\NMSim\01 SITES", des[:-4], r"Input_Data\01_ELEVATION", "elevation_m*.tif"))[0]
# utm_EPSG = "EPSG:" + str(26900 + int(elev_path.split("_")[-1][3:-4]))

# create a 100x100 point grid over the study area extent
x_coords = np.linspace(xmin, xmax, 200)
y_coords = np.linspace(ymin, ymax, 200)
xs, ys = np.meshgrid(x_coords, y_coords)
coords = [(x, y) for x, y in zip(np.ravel(xs), np.ravel(ys))]

# sample the raster to glean elevations!
with rasterio.open(elev_path) as ras:
    zraw = np.array([z[0] for z in ras.sample(coords)])

# clean up the elevation values
zs = zraw.reshape(len(y_coords), len(x_coords)) # CHECK FOR ORDER!
zs = zs.astype('float')
zs[zs == 32767] = np.nan

gdf = gpd.read_file(annotations)
gdf = gdf[gdf["note"] != "Too few points"] # these geometries are 2D
gdf = gdf[gdf["note"] != "No SPL data"] # these geometries are 2D
gdf["color"] = [Color_Map[i] for i in (gdf["valid"].astype('int') + gdf["audible"].astype('int'))]

gdf = gdf.loc[gdf["color"] == "skyblue", :]
gdf

##### Load the interactive plot

In [ ]:
site_coords = np.append(site_coords, 500)
site_coords

In [ ]:
# interactivity 'magic'
%matplotlib qt 

fig = plt.figure(figsize=(8, 8))
ax = plt.axes(projection='3d')
ax.set_box_aspect((1.0, 1.0, 0.25))

for _, row in gdf.iterrows():
    ax.plot3D(*np.array([point for point in row.geometry.coords]).T, 
              color=row["color"], alpha=0.5)

ax.plot3D(*site_coords, ls="", marker="o", markersize=5, color="magenta")
    
ax.axis(xmin=xmin,xmax=xmax,
        ymin=ymin,ymax=ymax)

ax.contour3D(xs, ys, zs, 50, cmap='binary', zorder=-1, alpha=0.2)

ax.set_zlim([0.9*np.nanmin(zs), 1.1*np.nanmax(zs)])

# Hide grid lines
ax.grid(False)

plt.show()